<a href="https://colab.research.google.com/github/olgOk/Adaptive_QNG/blob/main/code/NaturalGradientWithLineSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python3 -m pip install -q qiskit

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.3/241.3 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 KB 8.2 MB/s eta 0:00:00


In [3]:
from collections.abc import Iterable
from typing import List, Tuple, Callable, Optional, Union
import functools
import numpy as np

from qiskit.circuit.quantumcircuit import _compare_parameters
from qiskit.circuit import ParameterVector, ParameterExpression
from qiskit.utils import optionals as _optionals

from qiskit.opflow.operator_base import OperatorBase
from qiskit.opflow.list_ops.list_op import ListOp
from qiskit.opflow.list_ops.composed_op import ComposedOp
from qiskit.opflow.state_fns.circuit_state_fn import CircuitStateFn
from qiskit.opflow.gradients.circuit_gradients import CircuitGradient
from qiskit.opflow.gradients.circuit_qfis import CircuitQFI
from qiskit.opflow.gradients.gradient import Gradient
from qiskit.opflow.gradients.gradient_base import GradientBase
from qiskit.opflow.gradients.qfi import QFI

# Error tolerance variable
ETOL = 1e-8
# Cut-off ratio for small singular values for least square solver
RCOND = 1e-2

In [4]:
class LineSearchNaturalGradient(GradientBase):
  """Quantum Natural Gradient with Line Search.
  """

  def __init__(
      self,
      grad_method: Union[str, CircuitGradient] = "lin_comb",
      qfi_method: Union[str, CircuitQFI] = "lin_comb_full",
      **kwargs,
  ):
    """
    Args:
      grad_method: The method used to compute the state gradient. Can be either
                  ``'param_shift'`` or ``'lin_comb'`` or ``'fin_diff'``.
      qfi_method: The method used to compute the QFI. Can be either
                  ``'lin_comb_full'`` or ``'overlap_block_diag'`` or ``'overlap_diag'``.
    """
    super().__init__(grad_method)

    self._qfi_method = QFI(qfi_method)
    # is this the tolerance parameter?
    self._epsilon = kwargs.get("epsilon", 1e-3) 

  def convert(
      self,
      operator: OperatorBase,
      params: Optional[
          Union[ParameterVector, ParameterExpression, List[ParameterExpression]]
        ] = None,
    ) -> OperatorBase:
      """
      Args:
        operator: The operator we are taking the gradient of.
        params: The parameters we are taking the gradient with respect to. If not explicitly
                passed, they are inferred from the operator and sorted by name.
      Returns:
        An operator whose evaluation yields the NaturalGradient.
      Raises:
        TypeError: If ``operator`` does not represent an expectation value or the quantum
                state is not ``CircuitStateFn``.
          ValueError: If ``params`` contains a parameter not present in ``operator``.
          ValueError: If ``operator`` is not parameterized.
      """
      if not isinstance(operator, ComposedOp):
        if not (isinstance(operator, ListOp) and len(operator.oplist) == 1):
          raise TypeError(
            "Please provide the operator either as ComposedOp or as ListOp of "
            "a CircuitStateFn potentially with a combo function."
          )

      if not isinstance(operator[-1], CircuitStateFn):
        raise TypeError(
          "Please make sure that the operator for which you want to compute "
          "Quantum Fisher Information represents an expectation value or a "
          "loss function and that the quantum state is given as "
          "CircuitStateFn."
          )
      if len(operator.parameters) == 0:
          raise ValueError("The operator we are taking the gradient of is not parameterized!")
      if params is None:
          params = sorted(operator.parameters, key=functools.cmp_to_key(_compare_parameters))
      if not isinstance(params, Iterable):
          params = [params]
      # Instantiate the gradient
      grad = Gradient(self._grad_method, epsilon=self._epsilon).convert(operator, params)

      # Instantiate the QFI metric which is used to re-scale the gradient
      metric = self._qfi_method.convert(operator[-1], params) * 0.25

      def combo_fn(x):
        print(x)
        return self.nat_grad_combo_fn(x)

      # Define the ListOp which combines the gradient and the QFI according to the combination
      # function defined above.
      return ListOp([grad, metric], combo_fn=combo_fn)

 
  def nat_grad_combo_fn(x) -> np.ndarray:
      """
      Natural Gradient Function Implementation.
      Args:
        x: Iterable consisting of Gradient, Quantum Fisher Information.
            regularization: Regularization method.
      Returns:
        Natural Gradient.
      Raises:
          ValueError: If the gradient has imaginary components that are non-negligible.
      """
      gradient = x[0]
      metric = x[1]

      if np.amax(np.abs(np.imag(gradient))) > ETOL:
        raise ValueError(
          "The imaginary part of the gradient are non-negligible. The largest absolute "
          f"imaginary value in the gradient is {np.amax(np.abs(np.imag(gradient)))}. "
          "Please increase the number of shots."
            )
        gradient = np.real(gradient)

        if np.amax(np.abs(np.imag(metric))) > ETOL:
            raise ValueError(
                "The imaginary part of the metric are non-negligible. The largest "
                "absolute imaginary value in the gradient is "
                f"{np.amax(np.abs(np.imag(metric)))}. Please "
                "increase the number of shots."
            )
        metric = np.real(metric)

        # Check if numerical instabilities lead to a metric which is not positive semidefinite
        w, v = np.linalg.eigh(metric)

        if not all(ew >= (-1) * ETOL for ew in w):
          raise ValueError(
                    f"The underlying metric has at least one Eigenvalue < -{ETOL}. "
                    f"The smallest Eigenvalue is {np.amin(w)} "
                    "Please use a regularized least-square solver for this problem or "
                    "increase the number of backend shots.",
                    )
        if not all(ew >= 0 for ew in w):
          # If not all eigenvalues are non-negative, set them to a small positive
          # value
          w = [max(ETOL, ew) for ew in w]
          # Recompose the adapted eigenvalues with the eigenvectors to get a new metric
          metric = np.real(v @ np.diag(w) @ np.linalg.inv(v))
      
      nat_grad = LineSearchNaturalGradient._line_search_solver(metric, gradient)

      nat_grad = np.linalg.lstsq(metric, gradient, rcond=RCOND)[0]
      return nat_grad

  
  def _line_search_solver(
        metric: np.ndarray,
        gradient: np.ndarray,
    )-> np.ndarray:

    from sklearn.ensemble import GradientBoostingRegressor

    params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error"}

    reg = ensemble.GradientBoostingRegressor(**params)

    def reg_method(a, c, alpha):
            reg.set_params(alpha=alpha)
            if normalize:
                reg.fit(StandardScaler().fit_transform(a), c)
            else:
                reg.fit(a, c)
            return reg.coef_

    something = reg_method(metric, gradient, alpha=0.1)

    print(something)


  def back_tracking_lin_search(cal_obj, cal_obj_grad, param, beta, alpha, max_iter=10):
    """Implementation of the Backtracking Line Search with Armijo's rule.
    """
    
    grad = cal_obj_grad(param)
    obj = cal_obj(param)
    
    for i in range(max_iter):
        tmp_beta = beta/2**i
        tmp_param = param - tmp_beta*grad
        tmp_obj = cal_obj(tmp_param)
        
        armijo_cond = alpha*tmp_beta*np.linalg.norm(grad)**2
        
        print(tmp_beta, tmp_param, obj, tmp_obj, armijo_cond)
        
        if abs(obj - tmp_obj) >= armijo_cond:
            return tmp_param
        
    return tmp_param
            





In [8]:
from qiskit.opflow import X, Z, I, StateFn, CircuitStateFn, SummedOp
from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit import QuantumCircuit, QuantumRegister, Parameter, ParameterVector, ParameterExpression

H_model = (0.4 * I ^ Z) + (0.4 * Z ^ I) + (0.2 * X ^ X)

# constract the ansatz
theta1 = Parameter('theta1')
theta2 = Parameter('theta2')
q = QuantumRegister(2)
qc = QuantumCircuit(q)
qc.ry(theta1, q[0])
qc.ry(theta2, q[1])
qc.cx(q[0],q[1])
params = [theta1, theta2]

op = ~StateFn(H_model) @ CircuitStateFn(primitive=qc, coeff=1.)

# define and compute the QFI
state = CircuitStateFn(primitive=qc, coeff=1.)
qfi = QFI(qfi_method='lin_comb_full').convert(operator=state, params=params)

values_dict = {theta1:-0.1, theta2:-0.2}

# Assign the parameters and evaluate the QFI
qfi_result = qfi.assign_parameters(values_dict).eval()
# print('full QFI \n', 0.25*np.real(np.array(qfi_result)))

In [ ]:
nat_grad = LineSearchNaturalGradient(grad_method='lin_comb', qfi_method='lin_comb_full').convert(operator=op, params=params)

# Assign the parameters and evaluate the gradient
nat_grad_result = nat_grad.assign_parameters(values_dict).eval()
print('Natural gradient computed with linear combination of unitaries', nat_grad_result)